# Demo Notebook

A notebook describing the basics for connecting via python to oracle. 

**First, ensure that you installed cx_Oracle with:**

`python -m pip install cx_Oracle --upgrade`

## Get the libraries

In [1]:
import cx_Oracle
import random
from datetime import datetime, timedelta

## Get Basic Client

see here: https://www.oracle.com/database/technologies/instant-client/winx64-64-downloads.html

In [2]:
cx_Oracle.init_oracle_client('/opt/oracle/instantclient_23_3') #add here the path to the instantclient

## Set the credentials

In [3]:
username="a11907564"  #db user name
password="dbs24" #db password 
con_string="oracle19.cs.univie.ac.at:1521/orclcdb"

## Connect

In [4]:
#consider to use with in order to avoid open connections
connection = cx_Oracle.connect(user=username, password=password,
                               dsn=con_string,
                               encoding="UTF-8")

Check if connection is now up!

In [57]:
#create a table
curs=connection.cursor()
create_stat="create table myTestTable(myId INTEGER)" #just an example: always use a PK
curs.execute(create_stat)
connection.commit()

DatabaseError: ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/

In [58]:
#insert into
myIdValue=4
create_stat="insert into myTestTable (myId) VALUES (:myIdValue)"
curs.execute(create_stat,myIdValue=myIdValue)
connection.commit()

In [59]:
#select
curs.execute("select * from myTestTable")
res = curs.fetchall()

In [60]:
str(res)  #result

'[(4,), (4,), (4,)]'

In [61]:
[x[0] for x in res]

[4, 4, 4]

# Read from CSV

In [62]:
import pandas as pd

In [63]:
example_data=pd.read_csv("ExampleData.csv",sep=";")

In [64]:
example_data.head()

,Id,Name
0,9,Ann
1,10,Bob
2,11,Carol


In [65]:
#create a table
create_stat="create table myTestTable2(myId INTEGER,myName VARCHAR(30))" #just an example: always use a PK
curs.execute(create_stat)
connection.commit()

DatabaseError: ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/

In [66]:
def add_tuple(m_tuple):
    create_stat="insert into myTestTable2 (myId,myName) VALUES (:myId,:myName)"
    curs.execute(create_stat,myId=m_tuple.Id,myName=m_tuple.Name)
    connection.commit()

In [67]:
example_data.apply(lambda x: add_tuple(x),axis=1)  #use for instead of apply if that is easier for your!
print("done with inserting....")

done with inserting....


In [68]:
curs.execute("select * from myTestTable2")
res = curs.fetchall()
res

[(9, 'Ann'),
 (10, 'Bob'),
 (11, 'Carol'),
 (9, 'Ann'),
 (10, 'Bob'),
 (11, 'Carol'),
 (9, 'Ann'),
 (10, 'Bob'),
 (11, 'Carol')]

# DB Triggers

execute the Triggers from Aufgabe 3-2.

Can be found in db_triggers.sql

In [7]:
import os


# Path to the SQL file
sql_file_path = 'db_trigger.sql'

try:
    # Read the SQL file
    with open(sql_file_path, 'r') as file:
        sql_commands = file.read()

    # Execute the SQL commands
    for command in sql_commands.split(';'):  # Split commands if needed
        command = command.strip()
        if command:  # Ignore empty commands
            curs.execute(command)
            print(f"Executed: {command}")

    # Commit changes
    connection.commit()
    print("All commands executed successfully.")

except Exception as e:
    print(f"Error: {e}")

Error: name 'curs' is not defined


# Aufgabe 3-3

1. Verbindung zur Datenbank

In [4]:
import cx_Oracle

# first wie want to establish a connection to the database (just in case this hasn't allready been done)

connection = cx_Oracle.connect(user=username, password=password,
                               dsn=con_string,
                               encoding="UTF-8")
curs = connection.cursor()

print("Verbindung erfolgreich!")



#then we want to make sure that the physical layout of the DB is established. The according Requests can be found in the File db_triggers.sql

# Path to the SQL file
sql_file_path = 'db_trigger.sql' 

try:
    # Read the SQL file
    with open(sql_file_path, 'r') as file:
        sql_commands = file.read()
        print(file)

    # Execute the SQL commands
    for command in sql_commands.split(';'):  # Split commands if needed
        command = command.strip()
        if command:  # Ignore empty commands
            try:
                curs.execute(command)
                print(f"Executed: {command}")
            except Exception as e:
                print(f"Error : {e}")

    # Commit changes
    connection.commit()
    print("All commands executed successfully.")

except Exception as e:
    print(f"Error: {e}")



Verbindung erfolgreich!
<_io.TextIOWrapper name='db_trigger.sql' mode='r' encoding='utf-8'>
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
Error : ORA-00955: name is already used by an existing object
Help: https://docs.oracle.c

In [10]:
sql_commands = [
    """
    CREATE OR REPLACE TRIGGER auto_increment_Kunden_Anbieter
    BEFORE INSERT ON Kunden_Anbieter
    FOR EACH ROW
    BEGIN
      SELECT Kunden_Anbieter_Seq.NEXTVAL INTO :NEW.ID FROM dual;
    END;
    """,
    """
    CREATE OR REPLACE TRIGGER update_vehicle_status
    AFTER INSERT ON Reservierungen
    FOR EACH ROW
    BEGIN
      UPDATE Fahrzeuge
      SET Status = 'Reserved'
      WHERE FahrzeugID = :NEW.FahrzeugID;
    END;
    """
]

# Execute each command
try:
    for command in sql_commands:
        curs.execute(command)
        print("Executed SQL command successfully.")
    connection.commit()
    print("All commands executed and committed successfully.")
except Exception as e:
    print(f"Error: {e}")


Executed SQL command successfully.
Executed SQL command successfully.
All commands executed and committed successfully.


2. Generierung von Testdaten


In [ ]:
# Kunden-Tabelle mit 100000 Einträgen befüllen - 100000 für Punkt 4 (Indexierung)

for i in range(1, 100001):
    kunden_id = i
    name = f'Kunde_{i}'
    adresse = f'Straße {random.randint(1, 100)}, Stadt {random.randint(1, 10)}'
    telefonnummer = f'01234567{random.randint(100, 999)}'
    email = f'kunde_{i}@example.com'
    try:
        curs.execute("""
            INSERT INTO Kunden (KundenID, Name, Adresse, Telefonnummer, Email)
            VALUES (:1, :2, :3, :4, :5)
        """, (kunden_id, name, adresse, telefonnummer, email))
    except Exception as e:
        print(f"Error: {e}")

connection.commit()
print("Kunden-Daten erfolgreich eingefügt!")


Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116313) violated
Help: https://docs.oracle.com/error-hel

In [17]:
# Fahrzeuge-Tabelle mit 500 Einträgen
for i in range(1, 501):
    fahrzeug_id = i
    kennzeichen = f'ABC-{i:04d}'
    modell = random.choice(['SUV', 'Kleinwagen', 'Limousine'])
    hersteller = random.choice(['Hersteller A', 'Hersteller B'])
    baujahr = random.randint(2000, 2023)
    status = random.choice(['verfügbar', 'vermietet', 'in Wartung'])
    try:
        curs.execute("""
            INSERT INTO Fahrzeuge (FahrzeugID, Kennzeichen, Modell, Hersteller, Baujahr, Status)
            VALUES (:1, :2, :3, :4, :5, :6)
        """, (fahrzeug_id, kennzeichen, modell, hersteller, baujahr, status))
    except Exception as e:
        print(f"Error: {e}")

connection.commit()
print("Fahrzeug-Daten erfolgreich eingefügt!")


Fahrzeug-Daten erfolgreich eingefügt!


In [15]:
# Reservierungen-Tabelle mit 10.000 Einträgen
for i in range(1, 10001):
    reservierungs_id = i
    kunden_id = random.randint(1, 1000)
    fahrzeug_id = random.randint(1, 500)
    startdatum = f'2025-01-{random.randint(1, 28)}'
    enddatum = f'2025-01-{random.randint(29, 31)}'
    kosten = round(random.uniform(50, 500), 2)
    try:
        curs.execute("""
            INSERT INTO Reservierungen (ReservierungsID, KundenID, FahrzeugID, Startdatum, Enddatum, Kosten)
            VALUES (:1, :2, :3, TO_DATE(:4, 'YYYY-MM-DD'), TO_DATE(:5, 'YYYY-MM-DD'), :6)
        """, (reservierungs_id, kunden_id, fahrzeug_id, startdatum, enddatum, kosten))
    except Exception as e:
        print(f"Error: {e}")
    
connection.commit()
print("Reservierungs-Daten erfolgreich eingefügt!")


Error: ORA-00001: unique constraint (A11907564.SYS_C001116316) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116316) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116316) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116316) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116316) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116316) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116316) violated
Help: https://docs.oracle.com/error-help/db/ora-00001/
Error: ORA-00001: unique constraint (A11907564.SYS_C001116316) violated
Help: https://docs.oracle.com/error-hel

In [ ]:
# Fill the Anbieter table
print("Populating Anbieter table...")
for i in range(1, 101):  # 100 entries for Anbieter
    anbieter_id = i
    name = f"Anbieter_{i}"
    adresse = f"Straße {random.randint(1, 50)}, Stadt {random.randint(1, 10)}"
    telefonnummer = f"01234567{random.randint(100, 999)}"
    email = f"anbieter_{i}@example.com"
    try:
        curs.execute("""
            INSERT INTO Anbieter (AnbieterID, Name, Adresse, Telefonnummer, Email)
            VALUES (:1, :2, :3, :4, :5)
        """, (anbieter_id, name, adresse, telefonnummer, email))
    except Exception as e:
        print(f"Error inserting into Anbieter: {e}")

connection.commit()
print("Anbieter table populated successfully!")

In [ ]:
# Fill the Zusatzleistungen table
print("Populating Zusatzleistungen table...")
for i in range(1, 501):  # 500 entries for Zusatzleistungen
    leistungs_id = i
    reservierungs_id = random.randint(1, 10000)  # Assuming 10,000 reservations exist
    beschreibung = random.choice(["Zusatzversicherung", "Kindersitz", "Navigation"])
    kosten = round(random.uniform(5, 50), 2)
    try:
        curs.execute("""
            INSERT INTO Zusatzleistungen (LeistungsID, ReservierungsID, Beschreibung, Kosten)
            VALUES (:1, :2, :3, :4)
        """, (leistungs_id, reservierungs_id, beschreibung, kosten))
    except Exception as e:
        print(f"Error inserting into Zusatzleistungen: {e}")

connection.commit()
print("Zusatzleistungen table populated successfully!")

In [ ]:
# Fill the Kunden_Anbieter table
print("Populating Kunden_Anbieter table...")
for i in range(1, 1001):  # 1,000 entries for Kunden_Anbieter
    id_val = i
    kunden_id = random.randint(1, 1000)  # Assuming 1,000 customers exist
    anbieter_id = random.randint(1, 100)  # Assuming 100 providers exist
    try:
        curs.execute("""
            INSERT INTO Kunden_Anbieter (ID, KundenID, AnbieterID)
            VALUES (:1, :2, :3)
        """, (id_val, kunden_id, anbieter_id))
    except Exception as e:
        print(f"Error inserting into Kunden_Anbieter: {e}")

connection.commit()
print("Kunden_Anbieter table populated successfully!")

In [ ]:
print("Populating Transaktionen table...")

# Assume 10,000 reservations exist, so we'll create 10,000 transactions
for i in range(1, 10001):
    transaktions_id = i
    reservierungs_id = random.randint(1, 10000)  # Assuming 10,000 reservations
    betrag = round(random.uniform(50, 500), 2)  # Random amount between 50 and 500
    zahlungsweise = random.choice(['Kreditkarte', 'Bar', 'Überweisung'])  # Random payment method
    zahlungsdatum = (datetime.now() - timedelta(days=random.randint(1, 365))).strftime('%Y-%m-%d')  # Random date within the last year

    try:
        # Insert into the Transaktionen table
        curs.execute("""
            INSERT INTO Transaktionen (TransaktionsID, ReservierungsID, Betrag, Zahlungsweise, Zahlungsdatum)
            VALUES (:1, :2, :3, :4, TO_DATE(:5, 'YYYY-MM-DD'))
        """, (transaktions_id, reservierungs_id, betrag, zahlungsweise, zahlungsdatum))
    except Exception as e:
        print(f"Error inserting into Transaktionen: {e}")

# Commit the transactions to the database
connection.commit()
print("Transaktionen table populated successfully!")

3. Sinnvolle Querries

Query 1: Durchschnittliche Kosten pro Fahrzeugmodell


In [17]:
query1 = """
SELECT Modell, AVG(Kosten) AS Durchschnittskosten
FROM Fahrzeuge
JOIN Reservierungen ON Fahrzeuge.FahrzeugID = Reservierungen.FahrzeugID
GROUP BY Modell
HAVING AVG(Kosten) > 200
"""
curs.execute(query1)
for row in curs.fetchall():
    print(f"Output: {row}")


Output: ('Kleinwagen', 274.40583735909826)
Output: ('Limousine', 273.5024405594406)
Output: ('SUV', 275.52748096074987)


Query 2: Top-Kunden mit den meisten Reservierungen

In [18]:
query2 = """
SELECT Kunden.Name, COUNT(Reservierungen.ReservierungsID) AS AnzahlReservierungen
FROM Kunden
JOIN Reservierungen ON Kunden.KundenID = Reservierungen.KundenID
GROUP BY Kunden.Name
ORDER BY AnzahlReservierungen DESC
FETCH FIRST 10 ROWS ONLY
"""
curs.execute(query2)
for row in curs.fetchall():
    print(row)


('Kunde_264', 20)
('Kunde_801', 20)
('Kunde_58', 19)
('Kunde_385', 19)
('Kunde_862', 19)
('Kunde_614', 19)
('Kunde_254', 19)
('Kunde_93', 18)
('Kunde_682', 18)
('Kunde_737', 18)


Query 3: Fahrzeuge mit aktuellen Reservierungen

In [19]:
query3 = """
SELECT Fahrzeuge.FahrzeugID, Fahrzeuge.Kennzeichen, Reservierungen.Startdatum, Reservierungen.Enddatum
FROM Fahrzeuge
JOIN Reservierungen ON Fahrzeuge.FahrzeugID = Reservierungen.FahrzeugID
WHERE Reservierungen.Startdatum <= SYSDATE AND Reservierungen.Enddatum >= SYSDATE
"""
curs.execute(query3)
for row in curs.fetchall():
    print(row)


(116, 'ABC-0116', datetime.datetime(2025, 1, 3, 0, 0), datetime.datetime(2025, 1, 30, 0, 0))
(421, 'ABC-0421', datetime.datetime(2025, 1, 5, 0, 0), datetime.datetime(2025, 1, 29, 0, 0))
(455, 'ABC-0455', datetime.datetime(2025, 1, 18, 0, 0), datetime.datetime(2025, 1, 31, 0, 0))
(303, 'ABC-0303', datetime.datetime(2025, 1, 20, 0, 0), datetime.datetime(2025, 1, 31, 0, 0))
(427, 'ABC-0427', datetime.datetime(2025, 1, 2, 0, 0), datetime.datetime(2025, 1, 29, 0, 0))
(2, 'ABC-0002', datetime.datetime(2025, 1, 19, 0, 0), datetime.datetime(2025, 1, 29, 0, 0))
(428, 'ABC-0428', datetime.datetime(2025, 1, 3, 0, 0), datetime.datetime(2025, 1, 30, 0, 0))
(49, 'ABC-0049', datetime.datetime(2025, 1, 5, 0, 0), datetime.datetime(2025, 1, 31, 0, 0))
(489, 'ABC-0489', datetime.datetime(2025, 1, 18, 0, 0), datetime.datetime(2025, 1, 31, 0, 0))
(478, 'ABC-0478', datetime.datetime(2025, 1, 12, 0, 0), datetime.datetime(2025, 1, 30, 0, 0))
(144, 'ABC-0144', datetime.datetime(2025, 1, 1, 0, 0), datetime.date

4. Neue Safe Procedure für das Updaten von Anbieter-Einträgen

In [22]:
procedure_code = """
CREATE OR REPLACE PROCEDURE UpdateAnbieterEmail (
    p_AnbieterID IN NUMBER,
    p_NewEmail IN VARCHAR2
) AS
BEGIN
    UPDATE Anbieter
    SET Email = p_NewEmail
    WHERE AnbieterID = p_AnbieterID;

    IF SQL%ROWCOUNT = 0 THEN
        RAISE_APPLICATION_ERROR(-20001, 'AnbieterID not found.');
    END IF;
END;
"""

try:
    # Execute the procedure creation
    curs.execute(procedure_code)
    print("Procedure 'UpdateAnbieterEmail' created successfully.")
except Exception as e:
    print(f"Error creating procedure: {e}")


Procedure 'UpdateAnbieterEmail' created successfully.


In [25]:
try:
    # Call the UpdateAnbieterEmail procedure
    curs.callproc('UpdateAnbieterEmail', [1, 'new_email@example.com'])
    print("Anbieter email updated successfully.")
except Exception as e:
    print(f"Error: {e}")

connection.commit()

curs.execute("SELECT * FROM Anbieter WHERE AnbieterID = 1")
print(curs.fetchall())


Anbieter email updated successfully.
[(1, 'Anbieter_1', 'Straße 35, Stadt 10', '01234567888', 'new_email@example.com')]


5. Neue Delete Procedure um Zusatzleistungen zu löschen

In [23]:
procedure_code = """
CREATE OR REPLACE PROCEDURE DeleteZusatzleistungen (
    p_ReservierungsID IN NUMBER,
    p_MaxKosten IN NUMBER
) AS
BEGIN
    DELETE FROM Zusatzleistungen
    WHERE ReservierungsID = p_ReservierungsID AND Kosten <= p_MaxKosten;

    IF SQL%ROWCOUNT = 0 THEN
        RAISE_APPLICATION_ERROR(-20002, 'No records found for the given parameters.');
    END IF;
END;
"""

try:
    # Execute the procedure creation
    curs.execute(procedure_code)
    print("Procedure 'DeleteZusatzleistungen' created successfully.")
except Exception as e:
    print(f"Error creating procedure: {e}")


Procedure 'DeleteZusatzleistungen' created successfully.


In [28]:
try:
    # Call the DeleteZusatzleistungen procedure
    curs.callproc('DeleteZusatzleistungen', [5940, 100])  # Example: ReservierungsID=10, MaxKosten=30
    print("Zusatzleistungen deleted successfully.")
except Exception as e:
    print(f"Error: {e}")

connection.commit()

curs.execute("SELECT * FROM Zusatzleistungen WHERE ReservierungsID = 10")
print(curs.fetchall())


Zusatzleistungen deleted successfully.
[]


6. Indexierung anhand der Kunden Tabelle

In [24]:
try:
    # Execute the EXPLAIN PLAN query
    curs.execute("""
        EXPLAIN PLAN FOR
        SELECT * FROM Kunden WHERE email = 'kunde_1000@example.com'
    """)
    # Fetch the plan results
    curs.execute("SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY())")
    plan = curs.fetchall()
    print("Execution plan without index:")
    for row in plan:
        print(row)
except Exception as e:
    print(f"Error: {e}")


Execution plan without index:
('Plan hash value: 2881942826',)
(' ',)
('----------------------------------------------------------------------------',)
('| Id  | Operation         | Name   | Rows  | Bytes | Cost (%CPU)| Time     |',)
('----------------------------------------------------------------------------',)
('|   0 | SELECT STATEMENT  |        |     1 |    68 |     5   (0)| 00:00:01 |',)
('|*  1 |  TABLE ACCESS FULL| KUNDEN |     1 |    68 |     5   (0)| 00:00:01 |',)
('----------------------------------------------------------------------------',)
(' ',)
('Predicate Information (identified by operation id):',)
('---------------------------------------------------',)
(' ',)
('   1 - filter("EMAIL"=\'kunde_1000@example.com\')',)


In [29]:
try:
    # Create an index on the 'email' column
    curs.execute("CREATE INDEX idx_email ON Kunden (email)")
    connection.commit()
    print("Index created successfully.")
except Exception as e:
    print(f"Error creating index: {e}")


Index created successfully.


In [31]:
try:
    # Execute the EXPLAIN PLAN query after adding the index
    curs.execute("""
        EXPLAIN PLAN FOR
        SELECT * FROM Kunden WHERE email = 'kunde_1000@example.com'
    """)
    # Fetch the plan results
    curs.execute("SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY())")
    plan = curs.fetchall()
    print("Execution plan with index:")
    for row in plan:
        print(row)
except Exception as e:
    print(f"Error: {e}")


Execution plan with index:
('Plan hash value: 2211476835',)
(' ',)
('-------------------------------------------------------------------------------------------------',)
('| Id  | Operation                           | Name      | Rows  | Bytes | Cost (%CPU)| Time     |',)
('-------------------------------------------------------------------------------------------------',)
('|   0 | SELECT STATEMENT                    |           |     1 |    68 |     2   (0)| 00:00:01 |',)
('|   1 |  TABLE ACCESS BY INDEX ROWID BATCHED| KUNDEN    |     1 |    68 |     2   (0)| 00:00:01 |',)
('|*  2 |   INDEX RANGE SCAN                  | IDX_EMAIL |     1 |       |     1   (0)| 00:00:01 |',)
('-------------------------------------------------------------------------------------------------',)
(' ',)
('Predicate Information (identified by operation id):',)
('---------------------------------------------------',)
(' ',)
('   2 - access("EMAIL"=\'kunde_1000@example.com\')',)


# Close Connection

In [89]:
curs.close()
connection.close()